In [1]:
%reset -sf

In [184]:
#!/usr/bin/env python3
import sys
import math, random
import functools, itertools, collections, heapq, bisect
from collections import Counter, defaultdict, deque

import matplotlib.pyplot as plt

abc = "abcdefghijklmnopqrstuvwxyz"
M9 = 10**9 + 7  # 998244353
yes, no = "YES", "NO"
d4 = [(1,0),(0,1),(-1,0),(0,-1)]
d8 = [
    (-1,-1),(-1,0),(-1,1),
    (0,-1),(0,0),(0,1),
    (1,-1),(1,0),(1,1)]
d6 = [(2,0),(1,1),(-1,1),(-2,0),(-1,-1),(1,-1)]  # hexagonal layout
MAXINT = sys.maxsize

In [247]:
arr = """
[N] [G]                     [Q]    
[H] [B]         [B] [R]     [H]    
[S] [N]     [Q] [M] [T]     [Z]    
[J] [T]     [R] [V] [H]     [R] [S]
[F] [Q]     [W] [T] [V] [J] [V] [M]
[W] [P] [V] [S] [F] [B] [Q] [J] [H]
[T] [R] [Q] [B] [D] [D] [B] [N] [N]
[D] [H] [L] [N] [N] [M] [D] [D] [B]
""".strip()

In [248]:
stack = [[] for _ in range(11)]

In [249]:
for brr in arr.split("\n")[::-1]:
    for i,x in enumerate(
        [brr[1], brr[5], brr[9], brr[13], brr[17], brr[21], brr[25], brr[29], brr[33]],
        start = 1
    ):
        if x != " ":
            stack[i].append(x)

In [251]:
# srr = """
# """.strip()

In [252]:
# srr = """
# """.strip()

In [253]:
for arr in srr.split("\n"):
    _,a,_,b,_,c = arr.split()
    a, b, c = int(a), int(b), int(c)
    rev = []
    for _ in range(a):
        x = stack[b].pop()
        rev.append(x)
    for x in rev[::-1]:
        stack[c].append(x)

In [254]:
print("".join(stack[x][-1] for x in range(1,10)))

QLFQDBBHM


In [255]:
# with open("/Users/hkmac/Downloads/input.txt") as f:
#     srr = f.read().strip()

# brr = srr.split("\n\n")

# xrr = \
# [
#     [
#         [
#             int(drr) 
#             for drr in crr.split("-")
#         ]
#         for crr in brr.split(",")
#     ]
#     for brr in srr.split("\n")
# ]

# brr = srr.split("\n")

In [245]:
# xrr

In [191]:
res = 0

In [1]:
# sum(sorted(brr)[-3:])

In [ ]:
def test(arr):
    res = 0
    return res